In [ ]:
!nvidia-smi

In [ ]:

!wget -O food11.zip https://www.dropbox.com/s/up5q1gthsz3v0dq/food-11.zip?dl=0


In [ ]:
! unzip food11.zip

In [ ]:
_exp_name = "sample"

In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
import torchvision.transforms as transforms
from torch.utils.data import Dataset, DataLoader
import torchvision.models as models
from tqdm import tqdm
from PIL import Image
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import time
import os
import random  
from torchvision.models import vgg16, resnet18, resnet50

myseed = 6666
torch.backends.cudnn.deterministic = True
torch.backends.cudnn.benchmark = False
np.random.seed(myseed)
torch.manual_seed(myseed)
random.seed(myseed)  # 新增：设置 random 的种子
if torch.cuda.is_available():
    torch.cuda.manual_seed_all(myseed)


In [ ]:
device = "cuda" if torch.cuda.is_available() else "cpu"
batch_size = 64
n_epochs = 16 
mixup_alpha = 0.4
patience = 7  

lr = 1e-4
weight_decay = 3e-3


In [ ]:
# 資料載入（請自行修改 root 路徑）

# 原始代码
test_tfm = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
])

train_tfm = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.RandomAffine(degrees=0, translate=(0.1, 0.1), scale=(0.8, 1.2)),
    transforms.RandomHorizontalFlip(),
    transforms.RandomPerspective(distortion_scale=0.2, p=0.5),
    transforms.TrivialAugmentWide(),
    transforms.ToTensor(),
    transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
])
class FoodDataset(Dataset):

    def __init__(self,path,tfm=test_tfm,files = None):
        super(FoodDataset).__init__()
        self.path = path
        self.files = sorted([os.path.join(path,x) for x in os.listdir(path) if x.endswith(".jpg")])
        if files != None:
            self.files = files

        self.transform = tfm

    def __len__(self):
        return len(self.files)

    def __getitem__(self,idx):
        fname = self.files[idx]
        im = Image.open(fname)
        im = self.transform(im)

        try:
            label = int(fname.split("/")[-1].split("_")[0])
        except:
            label = -1 # test has no label

        return im,label

# Construct train and valid and test datasets.
# The argument "loader" tells how torchvision reads the data.
train_set = FoodDataset("./train", tfm=train_tfm)
train_loader = DataLoader(train_set, batch_size=batch_size, shuffle=True, num_workers=0, pin_memory=True)
valid_set = FoodDataset("./valid", tfm=test_tfm)
valid_loader = DataLoader(valid_set, batch_size=batch_size, shuffle=False, num_workers=0, pin_memory=True)
test_set = FoodDataset("./test", tfm=test_tfm)
test_loader = DataLoader(test_set, batch_size=batch_size, shuffle=False, num_workers=0, pin_memory=True)


In [ ]:
def get_model(model_name):
    if model_name == 'vgg16':
        model = vgg16(weights='IMAGENET1K_V1')
        model.classifier[6] = nn.Linear(4096, 11)
    elif model_name == 'resnet18':
        model = resnet18(weights='IMAGENET1K_V1')
        model.fc = nn.Linear(model.fc.in_features, 11)
    elif model_name == 'resnet50':
        model = resnet50(weights='IMAGENET1K_V1')
        model.fc = nn.Linear(model.fc.in_features, 11)
    return model.to(device)


# train_and_evaluate version2

In [ ]:
# === 原本程式碼開始 ===
def train_and_evaluate(model_name):
    model = get_model(model_name)
    optimizer = torch.optim.Adam(model.parameters(), lr=lr, weight_decay=weight_decay)
    scheduler = torch.optim.lr_scheduler.CosineAnnealingLR(optimizer, T_max=n_epochs)
    criterion = nn.CrossEntropyLoss()

    best_acc = 0
    stale = 0

    train_losses, train_accs = [], []
    valid_losses, valid_accs = [], []

    # === 新增：記錄學習率與單次訓練時間 ===
    lrs = []
    epoch_times = []

    for epoch in range(n_epochs):
        model.train()
        train_loss = []
        train_acc_epoch = []

        start_time = time.time()  # === 新增 ===

        for imgs, labels in tqdm(train_loader, desc=f"[{model_name}] Epoch {epoch+1}"):
            imgs, labels = imgs.to(device), labels.to(device)

            # Mixup
            lam = np.random.beta(mixup_alpha, mixup_alpha)
            index = torch.randperm(imgs.size(0)).to(device)
            mixed_imgs = lam * imgs + (1 - lam) * imgs[index]
            labels_a, labels_b = labels, labels[index]

            logits = model(mixed_imgs)
            loss = lam * criterion(logits, labels_a) + (1 - lam) * criterion(logits, labels_b)

            optimizer.zero_grad()
            loss.backward()
            optimizer.step()

            acc = (logits.argmax(dim=-1) == labels).float().mean()
            train_loss.append(loss.item())
            train_acc_epoch.append(acc.item())

        train_losses.append(np.mean(train_loss))
        train_accs.append(np.mean(train_acc_epoch))

        # Validation
        model.eval()
        valid_loss, valid_acc_epoch = [], []
        with torch.no_grad():
            for imgs, labels in valid_loader:
                imgs, labels = imgs.to(device), labels.to(device)
                logits = model(imgs)
                loss = criterion(logits, labels)
                acc = (logits.argmax(dim=-1) == labels).float().mean()
                valid_loss.append(loss.item())
                valid_acc_epoch.append(acc.item())

        valid_loss_avg = np.mean(valid_loss)
        valid_acc_avg = np.mean(valid_acc_epoch)
        valid_losses.append(valid_loss_avg)
        valid_accs.append(valid_acc_avg)

        scheduler.step()
        # === 新增：記錄學習率與時間 ===
        lrs.append(optimizer.param_groups[0]['lr'])
        epoch_times.append(time.time() - start_time)

        print(f"Epoch {epoch+1}: Train acc={train_accs[-1]:.4f}, Valid acc={valid_acc_avg:.4f}")

        if valid_acc_avg > best_acc:
            best_acc = valid_acc_avg
            torch.save(model.state_dict(), f"{model_name}_best.ckpt")
            stale = 0
        else:
            stale += 1
            if stale > patience:
                print(f"Early stopping at epoch {epoch+1}")
                break

    # === 修改：return 增加學習率與訓練時間 ===
    return model, train_losses, train_accs, valid_losses, valid_accs, lrs, epoch_times
# === 原本程式碼結束 ===


# plot version2

In [ ]:
# === 原本程式碼開始 ===
from torchvision.models import vgg16, resnet18, resnet50
results = {}
for model_name in ['vgg16', 'resnet18', 'resnet50']:
    model, train_losses, train_accs, valid_losses, valid_accs, lrs, epoch_times = train_and_evaluate(model_name)
    results[model_name] = {
        "model": model,
        "train_losses": train_losses,
        "train_accs": train_accs,
        "valid_losses": valid_losses,
        "valid_accs": valid_accs,
        "lrs": lrs,
        "epoch_times": epoch_times
    }

    # === 原本繪圖：單一模型 Loss/Acc 圖 ===
    plt.figure(figsize=(10,6))
    epochs = range(1, len(train_losses) + 1)
    plt.plot(epochs, train_losses, label='Train Loss')
    plt.plot(epochs, valid_losses, label='Valid Loss')
    plt.plot(epochs, train_accs, label='Train Acc')
    plt.plot(epochs, valid_accs, label='Valid Acc')
    plt.xlabel('Epoch')
    plt.ylabel('Value')
    plt.title(f'{model_name} Loss and Accuracy')
    plt.legend()
    plt.grid(True)
    plt.savefig(f'/kaggle/working/{model_name}_plot.png')
    plt.show()
# === 原本程式碼結束 ===

# === 新增：兩兩模型比較圖（Loss 和 Accuracy） ===
from itertools import combinations

for model1, model2 in combinations(results.keys(), 2):
    plt.figure(figsize=(10,6))
    epochs1 = range(1, len(results[model1]['train_losses']) + 1)
    epochs2 = range(1, len(results[model2]['train_losses']) + 1)

    plt.plot(epochs1, results[model1]['valid_accs'], label=f'{model1} Valid Acc')
    plt.plot(epochs2, results[model2]['valid_accs'], label=f'{model2} Valid Acc')
    plt.plot(epochs1, results[model1]['valid_losses'], linestyle='--', label=f'{model1} Valid Loss')
    plt.plot(epochs2, results[model2]['valid_losses'], linestyle='--', label=f'{model2} Valid Loss')

    plt.xlabel('Epoch')
    plt.ylabel('Loss / Accuracy')
    plt.title(f'Comparison: {model1} vs {model2}')
    plt.legend()
    plt.grid(True)
    plt.savefig(f'/kaggle/working/compare_{model1}_vs_{model2}.png')
    plt.show()

# === 新增：學習率與訓練時間圖（所有模型同圖） ===
fig, ax1 = plt.subplots(figsize=(12,6))

for model_name, result in results.items():
    ax1.plot(result['lrs'], label=f'{model_name} LR')

ax1.set_xlabel('Epoch')
ax1.set_ylabel('Learning Rate')
ax1.set_title('Learning Rate Over Epochs')
ax1.legend()
ax1.grid(True)
plt.savefig('/kaggle/working/lr_plot.png')
plt.show()

# === 新增：訓練時間圖 ===
plt.figure(figsize=(12,6))
for model_name, result in results.items():
    plt.plot(range(1, len(result['epoch_times']) + 1), result['epoch_times'], label=f'{model_name} Time per Epoch')
plt.xlabel('Epoch')
plt.ylabel('Time (s)')
plt.title('Training Time per Epoch')
plt.legend()
plt.grid(True)
plt.savefig('/kaggle/working/time_plot.png')
plt.show()


In [ ]:
# === 對三個模型都進行測試，並輸出各自的 prediction CSV ===
for model_name in ['vgg16', 'resnet18', 'resnet50']:
    print(f"Generating prediction CSV for {model_name}...")
    test_model = results[model_name]['model']
    test_model.eval()
    preds = []

    for batch in tqdm(test_loader):
        imgs, _ = batch
        imgs = imgs.to(device)
        with torch.no_grad():
            logits = test_model(imgs)
        pred = logits.argmax(dim=-1).cpu().numpy()
        preds.extend(pred)

    df = pd.DataFrame({"Id": [f"{i:04d}" for i in range(len(preds))], "Category": preds})
    df.to_csv(f"prediction_{model_name}.csv", index=False)
